### Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("startup_funding.csv")


### Clean Startup Name and Investment Type

In [2]:

df.loc[(df["StartupName"].str.lower() == "ola cabs")|(df["StartupName"].str.lower() == "olacabs"), "StartupName"] = "Ola"
df.loc[df["StartupName"].str.lower().str.find("flipkart") > -1, "StartupName"] = "Flipkart"
df.loc[df["StartupName"].str.lower().str.find("paytm") > -1, "StartupName"] = "Paytm"
df.loc[df["StartupName"].str.lower().str.find("oyo rooms") > -1, "StartupName"] = "Oyo"
df.loc[df["StartupName"].str.lower().str.find("oyorooms") > -1, "StartupName"] = "Oyo"

df.loc[df["InvestmentType"] == "PrivateEquity", "InvestmentType"] = "Private Equity"
df.loc[df["InvestmentType"] == "Crowd funding", "InvestmentType"] = "Crowd Funding"
df.loc[df["InvestmentType"] == "SeedFunding", "InvestmentType"] = "Seed Funding"


In [3]:
df_cf_sf = df[(df["InvestmentType"] == "Crowd Funding") | (df["InvestmentType"] == "Seed Funding")]

### Only private equity fundings

In [4]:
df_pe = df[df["InvestmentType"] == "Private Equity"]

### Return Dictionary for Cities sorted

In [5]:
def dictionary_arr(arr):
    dictionary = dict()
    for cty in arr:
        if "/" in cty:
            j = cty.strip().split("/")
            for e in j:
                if e.strip() in ("Bangalore", "Mumbai", "Gurgaon", "Noida", "New Delhi"):
                    dictionary[e.strip()] = dictionary.get(e.strip(),0) + 1
        else:
            j = cty.strip()
            if j in ("Bangalore", "Mumbai", "Gurgaon", "Noida", "New Delhi"):
                dictionary[j] = dictionary.get(j,0) + 1
    return sorted(dictionary.items(),key = lambda x:x[1],reverse=True)

### Return Dictionary for Investor sorted

In [6]:
def dictionary_arr_investors(arr):
    dictionary = dict()
    for cty in arr:
        if "," in cty:
            j = cty.strip().split(",")
            for e in j:
                dictionary[e.strip()] = dictionary.get(e.strip(),0) + 1
        else:
            j = cty.strip()
            dictionary[j] = dictionary.get(j,0) + 1
    return sorted(dictionary.items(),key = lambda x:x[1],reverse=True)

### Call dictionary for city and investors

In [7]:
df.loc[df["CityLocation"]=="Delhi","CityLocation"] = "New Delhi"
arr = df[df["CityLocation"].notna()]["CityLocation"].str.title()
arri = df[df["InvestorsName"].notna()]["InvestorsName"]
arri_cf_sf = df_cf_sf[df_cf_sf["InvestorsName"].notna()]["InvestorsName"]
arri_pe = df_pe[df_pe["InvestorsName"].notna()]["InvestorsName"]

city_dict = dictionary_arr(arr)

investor_dict = dictionary_arr_investors(arri)

investor_cs_sf_dict = dictionary_arr(arri_cf_sf)

investor_pe_dict = dictionary_arr_investors(arri_pe)

# ANSWER 1: City with most times funded:

In [8]:
print(city_dict[0][0],city_dict[0][1])


Bangalore 637


# ANSWER 2:
### TOP 5 Investors most times funded:

In [9]:
for i in range(5):
    print(investor_dict[i][0],investor_dict[i][1])


Sequoia Capital 64
Accel Partners 53
Kalaari Capital 44
SAIF Partners 41
Indian Angel Network 40


In [10]:
df2 = df["InvestorsName"].str.split(",",expand = True)
df_cf_sf2 = df_cf_sf["InvestorsName"].str.split(",",expand = True)
df_pe2 = df_pe["InvestorsName"].str.split(",",expand = True)


In [11]:

for i in range(10):
    df2[i] = df2[i].str.strip()

for i in range(10):
    df_cf_sf2[i] = df_cf_sf2[i].str.strip()

for i in range(7):
    df_pe2[i] = df_pe2[i].str.strip()


In [12]:
ndf_pe = pd.concat([df_pe,df_pe2],axis = 1)

In [13]:
fdf_pe = pd.melt(ndf_pe,id_vars=["StartupName"],value_vars=[0,1,2,3,4,5,6])

In [14]:
fdf_pe = fdf_pe[fdf_pe["value"].str.lower().str.find("undisclosed investors")==-1]

In [15]:
fdf_pe = fdf_pe.drop_duplicates(subset=["value","StartupName"])

In [16]:
fdf_pe = fdf_pe.dropna()

# ANSWER 5:
### Top 5 investors from private equity most times funded unique startups

In [17]:
fdf_pe = fdf_pe[fdf_pe["value"] != ""]
llfdf_pe = fdf_pe["value"].value_counts()[:5]
for ele_pe in llfdf_pe.index:
    print(ele_pe,llfdf_pe[ele_pe])

Sequoia Capital 45
Accel Partners 43
Kalaari Capital 35
Blume Ventures 27
SAIF Partners 24


In [18]:
ndf_cf_sf = pd.concat([df_cf_sf,df_cf_sf2],axis = 1)

In [19]:
fdf_cf_sf = pd.melt(ndf_cf_sf,id_vars=["StartupName"],value_vars=[0,1,2,3,4,5,6,7,8,9])

In [20]:
fdf_cf_sf = fdf_cf_sf[fdf_cf_sf["value"].str.lower().str.find("undisclosed investors")==-1]

In [21]:
fdf_cf_sf = fdf_cf_sf.drop_duplicates(subset=["value","StartupName"])

In [22]:
fdf_cf_sf = fdf_cf_sf.dropna()

# ANSWER 4:
### Top 5 investor most times funded unique startups in Seed and Crowd funding

In [23]:
fdf_cf_sf = fdf_cf_sf[fdf_cf_sf["value"] != ""]
llfdf_cf_sf = fdf_cf_sf["value"].value_counts()[:5]
for ele_cf_sf in llfdf_cf_sf.index:
    print(ele_cf_sf,llfdf_cf_sf[ele_cf_sf])

Indian Angel Network 33
Rajan Anandan 23
Anupam Mittal 16
LetsVenture 16
Group of Angel Investors 14


In [24]:
ndf = pd.concat([df,df2],axis=1)

In [25]:
fdf = pd.melt(ndf,id_vars=["StartupName"],value_vars=[0,1,2,3,4,5,6,7,8,9])

In [26]:
fdf = fdf[fdf["value"].str.lower().str.find("undisclosed investors")==-1]

In [27]:
fdf = fdf.drop_duplicates(subset=["value","StartupName"])

In [28]:
fdf = fdf.dropna()

# ANSWER 3:
### Top 5 Investors invested in unique startups overall round

In [29]:
fdf = fdf[fdf["value"] != ""]
ll = fdf["value"].value_counts()[:5]
for el in ll.index:
    print(el,ll[el])

Sequoia Capital 48
Accel Partners 47
Kalaari Capital 41
Indian Angel Network 40
Blume Ventures 36
